# Time evolution with time-dependent Hamiltonian

This notebook demonstrates how one can use the `TimeDependentExpMPOEvolution` to simulate the evolution under a time-dependent Hamiltonian: the model just reads out the `time` model parameter, and we do a full re-initialization of the model during each time step. (This could be optimized to e.g. just adjusting values in the final MPO, but generating the model is usually cheap compared to the actual evolution scaling as $\mathcal{O}(\chi^3)$.)

Note that this method is (currently) only correct to first order in `dt`; to acchieve higher order you need to adjust the algorithms in TeNPy to account for the time dependence.

In [1]:
import numpy as np
np.set_printoptions(precision=5, suppress=True, linewidth=120)

In [2]:
import tenpy
from tenpy.models.spins import SpinChain
tenpy.tools.misc.setup_logging(to_stdout="DEBUG")
import yaml

In [3]:
class MyTimeDepModel(SpinChain):
    def init_terms(self, model_params):
        model_params['Jz'] = 1./(model_params.get('time', 0.) + 0.01)
        super().init_terms(model_params)

In [6]:
params = """
model_class :  MyTimeDepModel
model_params :
    L : 14
    bc_MPS : finite

initial_state_params:
    method : lat_product_state
    product_state : [[up], [down]]

algorithm_class: TimeDependentExpMPOEvolution
algorithm_params:
    trunc_params:
        chi_max: 120
    dt : 0.05
    N_steps : 2
    compression_method: variational

final_time :  1.5
"""
sim_params = yaml.safe_load(params)


In [7]:
res = tenpy.run_simulation('RealTimeEvolution', **sim_params)

INFO    : new simulation
RealTimeEvolution
INFO    : output filename: None
INFO    : use trivial cache (keeps everything in RAM)
INFO    : RealTimeEvolution: reading 'final_time'=1.5
INFO    : RealTimeEvolution: reading 'algorithm_params'={'trunc_params': {'chi_max': 120}, 'dt': 0.05, 'N_steps': 2, 'compression_method': 'variational'}
INFO    : No cache due to chi=120 < cache_threshold_chi = 2000
INFO    : use trivial cache (keeps everything in RAM)
INFO    : RealTimeEvolution: reading 'model_class'='MyTimeDepModel'
INFO    : RealTimeEvolution: subconfig 'model_params'=Config(<2 options>, 'model_params')
INFO    : model_params: reading 'bc_MPS'='finite'
INFO    : MyTimeDepModel: set conserve to Sz
INFO    : model_params: reading 'L'=14
INFO    : model_params: setting 'Jz'=100.0
INFO    : model_params: reading 'Jz'=100.0
INFO    : RealTimeEvolution: subconfig 'initial_state_params'=Config(<2 options>, 'initial_state_params')
INFO    : initial_state_params: reading 'method'='lat_product_

In [ ]:
res['measurements']['energy_MPO']